In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
import random

from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Convert pixel values from 0 to 255 to [0,1] range
print(x_train.shape)
x_train = np.reshape(x_train, (-1, 28*28))/255.
print (x_train.shape)
x_test = np.reshape(x_test, (-1, 28*28))/255.

y_train = np.array([1 if y < 5 else 0 for y in y_train])
y_test = np.array([1 if y < 5 else 0 for y in y_test])

(60000, 28, 28)
(60000, 784)


In [3]:
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)
y_train.shape

(60000, 1)

In [4]:
# train fraction
frac = 0.8

# shuffle dataset
idx = np.random.randint(x_train.shape[0], size=len(x_train))
X = x_train[idx]
Y = y_train[idx]

train_stop = int(len(X) * frac)

X_ = X[:train_stop]
Y_ = Y[:train_stop]

X_v = X[train_stop:]
Y_v = Y[train_stop:]
Y_[0].shape

(1,)

In [5]:
# some hyperparams
training_epochs = 100

n_neurons_in_h1 = 10
n_neurons_in_h2 = 10
learning_rate = 0.01

n_features = len(X_[0])
labels_dim = 1

#############################

x = tf.placeholder(tf.float32, [None, n_features], name='input')
y = tf.placeholder(tf.float32, [None, 1], name='labels')
# x = X_.astype(np.float32)
# y = Y_.astype(np.float32)

# TF Variables are our neural net parameter tensors, we initialize them to random (gaussian) values in
# Layer1. Variables are allowed to be persistent across training epochs and updatable bt TF operations
W1 = tf.Variable(tf.truncated_normal([n_features, n_neurons_in_h1], mean=0, stddev=1 / np.sqrt(n_features)),
                 name='weights1')
# b1 = tf.Variable(tf.truncated_normal([n_neurons_in_h1], mean=0, stddev=1 / np.sqrt(n_features)), name='biases1')

# note the output tensor of the 1st layer is the activation applied to a
# linear transform of the layer 1 parameter tensors
# the matmul operation calculates the dot product between the tensors
y1 = tf.sigmoid((tf.matmul(x, W1)), name='activationLayer1')

# network parameters(weights and biases) are set and initialized (Layer2)
# W2 = tf.Variable(tf.random_normal([n_neurons_in_h1, n_neurons_in_h2], mean=0, stddev=1),
#                  name='weights2')
# # b2 = tf.Variable(tf.random_normal([n_neurons_in_h2], mean=0, stddev=1), name='biases2')
# # activation function(sigmoid)
# y2 = tf.sigmoid((tf.matmul(y1, W2)), name='activationLayer2')

# output layer weights and biases
Wo = tf.Variable(tf.random_normal([n_neurons_in_h1, labels_dim], mean=0, stddev=1 ),
                 name='weightsOut')
# bo = tf.Variable(tf.random_normal([labels_dim], mean=0, stddev=1), name='biasesOut')

# the sigmoid (binary softmax) activation is absorbed into TF's sigmoid_cross_entropy_with_logits loss
logits = (tf.matmul(y1, Wo))
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)

train = optimizer.apply_gradients(grads_and_vars)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for step in range(1):
    sess.run(train, feed_dict={x: X_, y: Y_})
    for gv in grads_and_vars:
        if gv[0] is not None:
            print(str(sess.run(gv[0], feed_dict={x: X_, y: Y_})) + " - " + str(gv[1].name))
        #print(step, sess.run(W), sess.run(b))
    weights = [v for v in tf.trainable_variables() if v.name == "weights1:0" or v.name == "weightsOut:0"]
    for weight in weights:
        val = sess.run(weight)
        print(weight.name + " - " + str(val.shape) + " - " + str(val))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] - weights1:0
[[     0.]
 [-24591.]
 [     0.]
 [-24591.]
 [     0.]
 [-24591.]
 [     0.]
 [     0.]
 [     0.]
 [     0.]] - weightsOut:0
weights1:0 - (784, 10) - [[-0.06428923  0.03716642 -0.01484323 ...  0.01288267  0.0195029
   0.00454424]
 [ 0.03393454 -0.04771896 -0.00476133 ... -0.0210189  -0.01699767
  -0.00412214]
 [-0.02495923  0.00566365  0.00206114 ...  0.03200849  0.0121543
  -0.06042   ]
 ...
 [ 0.01520554 -0.01146381 -0.0569476  ...  0.01426436  0.02707958
  -0.00961968]
 [-0.05049045 -0.00311015  0.03992454 ...  0.02669788 -0.03212445
   0.00530215]
 [-0.01965043 -0.04982369  0.01306009 ...  0.00999025  0.0236108
   0.00682935]]
weightsOut:0 - (10, 1) - [[-82.13977 ]
 [-77.

In [31]:
import tensorflow as tf
import numpy as np

x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = "Wvar")
b = tf.Variable(tf.zeros([1]), name = "Bvar")
y = W * x_data + b

loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
grads_and_vars = optimizer.compute_gradients(loss)

train = optimizer.apply_gradients(grads_and_vars)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for step in range(1):
    sess.run(train)
    for gv in grads_and_vars:
        if gv[0] is not None:
            print(str(sess.run(gv[0])) + " - " + str((gv[1].name)))

[0.13023472] - Wvar_1:0
[0.40048462] - Bvar_1:0
